# CBOW class
* ./ch04/cbow.py

In [ ]:
import sys
sys.path.append('..')
from common.np import *  # import numpy as np
from common.layers import Embedding
from ch04.negative_sampling_layer import NegativeSamplingLoss


class CBOW:
    def __init__(self, vocab_size, hidden_size, window_size, corpus):
        V, H = vocab_size, hidden_size

        # 가중치 초기화
        W_in = 0.01 * np.random.randn(V, H).astype('f')
        W_out = 0.01 * np.random.randn(V, H).astype('f')

        # 계층 생성
        self.in_layers = []
        for i in range(2 * window_size):
            layer = Embedding(W_in)  # Embedding 계층 사용
            self.in_layers.append(layer)
        self.ns_loss = NegativeSamplingLoss(W_out, corpus, power=0.75, sample_size=5)

        # 모든 가중치와 기울기를 배열에 모은다.
        layers = self.in_layers + [self.ns_loss]
        self.params, self.grads = [], []
        for layer in layers:
            self.params += layer.params
            self.grads += layer.grads

        # 인스턴스 변수에 단어의 분산 표현을 저장한다.
        self.word_vecs = W_in

    def forward(self, contexts, target):
        h = 0
        for i, layer in enumerate(self.in_layers):
            h += layer.forward(contexts[:, i])
        h *= 1 / len(self.in_layers)
        loss = self.ns_loss.forward(h, target)
        return loss

    def backward(self, dout=1):
        dout = self.ns_loss.backward(dout)
        dout *= 1 / len(self.in_layers)
        for layer in self.in_layers:
            layer.backward(dout)
        return None


# fit method
* ./common/trainer.py

In [ ]:
# 훈련할 때 쓰는 method
def fit(self, x, t, max_epoch=10, batch_size=32, max_grad=None, eval_interval=20):
    data_size = len(x)
    max_iters = data_size // batch_size
    self.eval_interval = eval_interval
    model, optimizer = self.model, self.optimizer
    total_loss = 0
    loss_count = 0

    start_time = time.time()
    for epoch in range(max_epoch):
        # 뒤섞기
        idx = numpy.random.permutation(numpy.arange(data_size))
        x = x[idx]
        t = t[idx]

        for iters in range(max_iters):
            batch_x = x[iters*batch_size:(iters+1)*batch_size]
            batch_t = t[iters*batch_size:(iters+1)*batch_size]

            # 기울기 구해 매개변수 갱신
            loss = model.forward(batch_x, batch_t)
            model.backward()
            params, grads = remove_duplicate(model.params, model.grads)  # 공유된 가중치를 하나로 모음
            if max_grad is not None:
                clip_grads(grads, max_grad)
            optimizer.update(params, grads)
            total_loss += loss
            loss_count += 1

            # 평가
            if (eval_interval is not None) and (iters % eval_interval) == 0:
                avg_loss = total_loss / loss_count
                elapsed_time = time.time() - start_time
                print('| 에폭 %d |  반복 %d / %d | 시간 %d[s] | 손실 %.2f'
                        % (self.current_epoch + 1, iters + 1, max_iters, elapsed_time, avg_loss))
                self.loss_list.append(float(avg_loss))
                total_loss, loss_count = 0, 0

        self.current_epoch += 1

# plot method
* ./common/trainer.py

In [ ]:
# 그림 그릴 때 쓰는 method
def plot(self, ylim=None):
    x = numpy.arange(len(self.loss_list))
    if ylim is not None:
        plt.ylim(*ylim)
    plt.plot(x, self.loss_list, label='train')
    plt.xlabel('반복 (x' + str(self.eval_interval) + ')')
    plt.ylabel('손실')
    plt.show()

# ./ch04/train.py

In [ ]:
# coding: utf-8
import sys
sys.path.append('..')
import numpy as np
from common import config
# GPU에서 실행하려면 아래 주석을 해제하세요(CuPy 필요).
# ===============================================
# config.GPU = True
# ===============================================
import pickle
from common.trainer import Trainer
from common.optimizer import Adam
from cbow import CBOW
from ch04.skip_gram import SkipGram
from common.util import create_contexts_target, to_cpu, to_gpu
from dataset import ptb

In [ ]:
# 하이퍼파라미터 설정
window_size = 5
hidden_size = 100
batch_size = 100
max_epoch = 10

# 데이터 읽기
corpus, word_to_id, id_to_word = ptb.load_data('train')
vocab_size = len(word_to_id)

contexts, target = create_contexts_target(corpus, window_size)
if config.GPU:
    contexts, target = to_gpu(contexts), to_gpu(target)

# 모델 등 생성
model = CBOW(vocab_size, hidden_size, window_size, corpus)
# model = SkipGram(vocab_size, hidden_size, window_size, corpus)
optimizer = Adam()
trainer = Trainer(model, optimizer)

# 학습 시작
trainer.fit(contexts, target, max_epoch, batch_size)     # 입력데이터 context와 라벨 target이 필요
trainer.plot()

# 나중에 사용할 수 있도록 필요한 데이터 저장
word_vecs = model.word_vecs                              # word_vecs: 훈련이 끝났을 때 input 벡터들
if config.GPU:
    word_vecs = to_cpu(word_vecs)
params = {}                                              # 공딕셔너리
params['word_vecs'] = word_vecs.astype(np.float16)
params['word_to_id'] = word_to_id
params['id_to_word'] = id_to_word
pkl_file = 'cbow_params.pkl'  # or 'skipgram_params.pkl'
with open(pkl_file, 'wb') as f:
    pickle.dump(params, f, -1)


# ./ch04/eval.py
얼마나 단어의 의미를 얼마나 잘 담고있는지 평가 (evaluate)

벡터의 덧셈과 뺄셈으로 **유추문제**를 풀 수 있다. <br>
코사인 유사도... 측정 <br>
코사인 유사도 가장 높은 것들을 뽑는 것 <br>
* 벡터 : 단어에 대응하는 벡터(행)

* 의미 뿐만 아니라 문법적인 요소
> * 시제
> * 복수
> * 비교 <br>

... 이런 것 까지 담고있음.

In [ ]:
import sys
sys.path.append('..')
from common.util import most_similar, analogy
import pickle


pkl_file = 'cbow_params.pkl'
# pkl_file = 'skipgram_params.pkl'

with open(pkl_file, 'rb') as f: # 파일 불러올거야
    params = pickle.load(f)
    word_vecs = params['word_vecs']
    word_to_id = params['word_to_id']
    id_to_word = params['id_to_word']

# 가장 비슷한(most similar) 단어 뽑기
querys = ['you', 'year', 'car', 'toyota']
for query in querys:
    most_similar(query, word_to_id, id_to_word, word_vecs, top=5) # 물어보는 단어(query)를 word_vecs 각 행에 내적...

# 유추(analogy) 작업
print('-'*50)
analogy('king', 'man', 'queen',  word_to_id, id_to_word, word_vecs)
analogy('take', 'took', 'go',  word_to_id, id_to_word, word_vecs)
analogy('car', 'cars', 'child',  word_to_id, id_to_word, word_vecs) # children
analogy('good', 'better', 'bad',  word_to_id, id_to_word, word_vecs) # worse가 나오길 희망하지만 말뭉치가 작아서 등장빈도가 낮아서 다른 비교형 단어들만 나옴


# analogy 함수
* ./common/util.py

In [ ]:
def analogy(a, b, c, word_to_id, id_to_word, word_matrix, top=5, answer=None): #top: 상위 n개 출력, answer: 주어진 c와 answer와의 유사도 출력
    for word in (a, b, c):
        if word not in word_to_id:
            print('%s(을)를 찾을 수 없습니다.' % word)
            return

    print('\n[analogy] ' + a + ':' + b + ' = ' + c + ':?')
    a_vec, b_vec, c_vec = word_matrix[word_to_id[a]], word_matrix[word_to_id[b]], word_matrix[word_to_id[c]]
    query_vec = b_vec - a_vec + c_vec
    query_vec = normalize(query_vec)

    similarity = np.dot(word_matrix, query_vec)

    if answer is not None:
        print("==>" + answer + ":" + str(np.dot(word_matrix[word_to_id[answer]], query_vec)))

    count = 0
    for i in (-1 * similarity).argsort(): # argsort() 오름차순 정렬 후 해당하는 인덱스, -1의 의미: 내림차순
        if np.isnan(similarity[i]): # nan이면 pass
            continue
        if id_to_word[i] in (a, b, c): # a,b,c들어가면 안되므로 걔네일시 pass
            continue
        print(' {0}: {1}'.format(id_to_word[i], similarity[i]))

        count += 1
        if count >= top: # top의 숫자를 만족하는 순간 끝내
            return

# normalize 함수
* ./common/util.py

In [ ]:
def normalize(x):
    if x.ndim == 2:
        s = np.sqrt((x * x).sum(1))
        x /= s.reshape((s.shape[0], 1))
    elif x.ndim == 1:
        s = np.sqrt((x * x).sum())
        x /= s
    return x


# ./ch04/skip_gram.py

In [ ]:
import sys
sys.path.append('..')  # 부모 디렉터리의 파일을 가져올 수 있도록 설정
from common.layers import *
from ch04.negative_sampling_layer import NegativeSamplingLoss


class SkipGram:
    def __init__(self, vocab_size, hidden_size, window_size, corpus):
        V, H = vocab_size, hidden_size
        rn = np.random.randn

        # 가중치 초기화
        W_in = 0.01 * rn(V, H).astype('f')
        W_out = 0.01 * rn(V, H).astype('f')

        # 계층 생성
        self.in_layer = Embedding(W_in)
        self.loss_layers = []
        for i in range(2 * window_size):
            layer = NegativeSamplingLoss(W_out, corpus, power=0.75, sample_size=5)
            self.loss_layers.append(layer)

        # 모든 가중치와 기울기를 리스트에 모은다.
        layers = [self.in_layer] + self.loss_layers
        self.params, self.grads = [], []
        for layer in layers:
            self.params += layer.params
            self.grads += layer.grads

        # 인스턴스 변수에 단어의 분산 표현을 저장한다.
        self.word_vecs = W_in

    def forward(self, contexts, target):
        h = self.in_layer.forward(target)

        loss = 0
        for i, layer in enumerate(self.loss_layers):
            loss += layer.forward(h, contexts[:, i])
        return loss

    def backward(self, dout=1):
        dh = 0
        for i, layer in enumerate(self.loss_layers):
            dh += layer.backward(dout)
        self.in_layer.backward(dh)
        return None

# ./ch04/train.py

In [ ]:
# coding: utf-8
import sys
sys.path.append('..')
import numpy as np
from common import config
# GPU에서 실행하려면 아래 주석을 해제하세요(CuPy 필요).
# ===============================================
# config.GPU = True
# ===============================================
import pickle
from common.trainer import Trainer
from common.optimizer import Adam
from cbow import CBOW
from ch04.skip_gram import SkipGram
from common.util import create_contexts_target, to_cpu, to_gpu
from dataset import ptb

In [ ]:
# 하이퍼파라미터 설정
window_size = 5
hidden_size = 100
batch_size = 100
max_epoch = 10

# 데이터 읽기
corpus, word_to_id, id_to_word = ptb.load_data('train')
vocab_size = len(word_to_id)

contexts, target = create_contexts_target(corpus, window_size)
if config.GPU:
    contexts, target = to_gpu(contexts), to_gpu(target)

# 모델 등 생성
# model = CBOW(vocab_size, hidden_size, window_size, corpus)
model = SkipGram(vocab_size, hidden_size, window_size, corpus) # 클래스를 skip_gram으로 수정해서 돌리기
optimizer = Adam()
trainer = Trainer(model, optimizer)

# 학습 시작
trainer.fit(contexts, target, max_epoch, batch_size)     # 입력데이터 context와 라벨 target이 필요
trainer.plot()

# 나중에 사용할 수 있도록 필요한 데이터 저장
word_vecs = model.word_vecs                              # word_vecs: 훈련이 끝났을 때 input 벡터들
if config.GPU:
    word_vecs = to_cpu(word_vecs)
params = {}                                              # 공딕셔너리
params['word_vecs'] = word_vecs.astype(np.float16)
params['word_to_id'] = word_to_id
params['id_to_word'] = id_to_word
pkl_file = 'skipgram_params.pkl'  # or 'cbow_params.pkl'
with open(pkl_file, 'wb') as f:
    pickle.dump(params, f, -1)


# trainer class
* ./common/trainer.py

In [ ]:
# coding: utf-8
import sys
sys.path.append('..')
import numpy
import time # 시간관련 library
import matplotlib.pyplot as plt
from common.np import *  # import numpy as np
from common.util import clip_grads


class Trainer:
    def __init__(self, model, optimizer): # model: skipgram이나 cbow의 instance, optimizer: adam이나 momentum 클래스의 instance
        self.model = model
        self.optimizer = optimizer
        self.loss_list = []
        self.eval_interval = None # evaluate interval, 평가를 하겠다는 의미
        self.current_epoch = 0

    def fit(self, x, t, max_epoch=10, batch_size=32, max_grad=None, eval_interval=20): # 훈련(학습)할 때 쓰는 method
        # x: data, t: label,
        # 1 epoch:데이터 전부 소진할 때까지 걸리는 시간, 따라서 epoch은 데이터 전부 소진하는 것을 그만큼 반복하겠다는 의미,
        # batch_size: 이 만큼씩 묶어서 학습, 한 번 학습할 때 소진하는 data의 개수,
        # max_grad: max_gradient 그래디언트 상한선 제한함으로써 기울기 폭발 예방,
        # eval_interval: 이만큼 학습할 때마다 평가해서 출력
        data_size = len(x)
        max_iters = data_size // batch_size # 1epoch 동안 학습한 횟수
        self.eval_interval = eval_interval
        model, optimizer = self.model, self.optimizer
        total_loss = 0
        loss_count = 0

        start_time = time.time() # start 시간 저장
        for epoch in range(max_epoch):
            # 뒤섞기(permutation), data가 섞여있지 않은 경우 (정렬돼있는 경우), 학습이 잘 안된다.
            # choice보다 개선됨.
            # 개선된 점: 랜덤 초이스에 의해 데이터 중복이 돼 어떤 데이터는 학습에 사용이 안될 수도 있는 문제를 permutation으로 해결
            idx = numpy.random.permutation(numpy.arange(data_size))
            x = x[idx]
            t = t[idx]

            for iters in range(max_iters):
                batch_x = x[iters*batch_size:(iters+1)*batch_size] # 슬라이싱해서 사용하겠다.
                batch_t = t[iters*batch_size:(iters+1)*batch_size]

                # 기울기 구해 매개변수 갱신
                loss = model.forward(batch_x, batch_t)
                model.backward()
                params, grads = remove_duplicate(model.params, model.grads)  # 공유된 가중치를 하나로 모음
                if max_grad is not None:
                    clip_grads(grads, max_grad)
                optimizer.update(params, grads)
                total_loss += loss
                loss_count += 1

                # 평가
                if (eval_interval is not None) and (iters % eval_interval) == 0:
                    avg_loss = total_loss / loss_count
                    elapsed_time = time.time() - start_time
                    print('| 에폭 %d |  반복 %d / %d | 시간 %d[s] | 손실 %.2f'
                          % (self.current_epoch + 1, iters + 1, max_iters, elapsed_time, avg_loss))
                    self.loss_list.append(float(avg_loss))
                    total_loss, loss_count = 0, 0

            self.current_epoch += 1

    def plot(self, ylim=None): # 그림 그릴 때 쓰는 method
        x = numpy.arange(len(self.loss_list))
        if ylim is not None:
            plt.ylim(*ylim)
        plt.plot(x, self.loss_list, label='train')
        plt.xlabel('반복 (x' + str(self.eval_interval) + ')')
        plt.ylabel('손실')
        plt.show()

# clip method
* ./common/util.py

In [ ]:
def clip_grads(grads, max_norm):
    total_norm = 0
    for grad in grads:
        total_norm += np.sum(grad ** 2)
    total_norm = np.sqrt(total_norm)

    rate = max_norm / (total_norm + 1e-6)
    if rate < 1:
        for grad in grads:
            grad *= rate